# Classical Phase Estimation (with QFT) module

The present notebook reviews the classical **Quantum Phase Estimation Algorithm**, which using the inverse of **Quantum Fourier Transform** ($\mathcal{QFT}^{\dagger}$), allows the estimation of the self values (phases) of a unitary operator. We will call this algorithm **CQPE** from now. The **CQPE** was developed into module *classical_qpe.py* of the package *PE*  of the present library *QQuantLib* (**QQuantLib/PE/classical_pe.py**). 

This algorithm was developed as a Python class called: *CQPE* inside the **QQuantLib/PE/classical_qpe.py**.

The present notebook and module are based on the following references:

* *Brassard, G., Hoyer, P., Mosca, M., & Tapp, A. (2000). Quantum amplitude amplification and estimation.AMS Contemporary Mathematics Series, 305. https://arxiv.org/abs/quant-ph/0005055v1*
* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*


In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus
myqlm_qpus = ["python", "c"]
# QLM qpus accessed using Qaptiva Access library
qlmass_qpus = ["qlmass_linalg", "qlmass_mps"]
# QLM qpus: Only in local Quantum Learning Machine
qlm_qpus = ["linalg", "mps"]

linalg_qpu = get_qpu(myqlm_qpus[1])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

## 1. Initial Inputs

For using the *ClassicalPE* python class inside the **QQuantLib/PE/classical_pe** module 2 mandatory inputs should be provided:

* 1. Initial State: this will be the initial quantum state needed for applying the Unitary Operator.
* 2. Unitary operator: the operator whose phase we want to estimate.

To explain how the *ClassicalPE* class works we are going to use the **IQPE** example from the Qiskit textbook:

https://qiskit.org/textbook/ch-labs/Lab04_IterativePhaseEstimation.html

https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb

We are going to reproduce the section **IPE example with a 1-qubit gate for U** from the Qiskit example.

In this section, we are going to create these two mandatory inputs!

In [ ]:
#Number Of Qubits
n_qbits = 1

### 1.1 Initial State

Initial State can be:
1. QLM QRoutine
2. QLM gate (or abstract gate)

In the Qiskit example, the initial state will be $|1\rangle$. 

The following cell creates this initial state:

In [ ]:
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state --svg

### 1.2 Unitary operator.

The unitary operator can be:

1. QLM QRoutine
2. QLM gate (or abstract gate)

In the Qiskit example, the unitary operator is the $\mathcal{S}$ gate. So the application of the unitary operator over the initial state will be:

$$\mathcal{S}|1\rangle = e^{i\frac{\pi}{2}}|1\rangle$$

In [ ]:
unitary_operator = qlm.PH(np.pi/2.0)  

In [ ]:
%qatdisplay unitary_operator --svg

## 2. Class CQPE: classical Quantum Phase Estimation algorithm step by step 

The problem of phase estimation can be stated as follows. Given an initial state $\left|\Psi \right\rangle$ and a phase operator $\mathcal{Q}$ such that:

$$\mathcal{Q}\left|\Psi \right\rangle = e^{2\pi i\lambda}\left|\Psi \right\rangle,$$

our goal is estimating $\lambda$.

So far we have the initial state $\left|\Psi \right\rangle = |1\rangle$ and the unitary operator whose phase we want to estimate $\mathcal{Q} = \mathcal{S}$. In this section, we are going to describe the class step by step and explain the basics of the **CQPE** algorithm.


### 2.1 Calling the *CQPE* class

The *CQPE* is inside **QQuantLib/PE/classical_qpe** module. 

In order to instantiate the class we need to provide a Python dictionary. Mandatory keys that the user should provide are, as explained in Section 1:

* initial_state : QLM routine or gate with an initial state $|\Psi\rangle$ was loaded (created in Section 1).
* unitary_operator :  QLM gate or routine with a Unitary operator ready to be applied to initial state $|\Psi\rangle$ (created in Section 1)

Additionally, there are other keys that are important for configuring the method:

* auxiliar_qbits_number : int. The number of auxiliary qubits ($m$) that are used for phase estimation (default 8). This number of qubits provided the precision of the phase estimation returned by the **QPE**: $\frac{1}{2^m}$
* qpu : QLM solver. If not provided class try to create a PyLinalg solver. It is recommended to give this key to the class.
* shots: int number of shots for the quantum job (default 10).
* complete : for shots different from zero all the possible states will be returned. 

In [ ]:
#Load Class
from QQuantLib.PE.classical_qpe import CQPE

In [ ]:
auxiliar_qbits_number = 2
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : auxiliar_qbits_number,
    #'complete': True,
    'shots':10
}
qft_pe = CQPE(**qft_pe_dict)

When the class is instantiated the properties *initial_state* and *q_gate* are overwritten with the given keys **initial_state** and **unitary_operator** respectively

In [ ]:
c = qft_pe.initial_state
%qatdisplay c --svg

In [ ]:
a = qft_pe.q_gate
%qatdisplay a --depth 2 --svg

### 2.2 Classical Quantum Phase Estimation algorithm

The classical quantum phase estimation algorithm (**QPE**) is composed of the following steps:

1. Allocate a number of qubits equal to the number of qubits needed by the initial state operator $\mathcal{Q}$-
2. Allocate $m$ additional qubits where the phase will be codified.
3. Apply a Hadamard gate to each auxiliary qubit
4. Each auxiliary qubit will apply a controlled $2^{i}$ power of the unitary operator to the initial state. The $i$ will be related to the position of the auxiliary qubit.
5. Apply the inverse of the Quantum Fourier Transform $\mathcal{QFT}^{\dagger}$ to the auxiliary qbits.
6. Measure the auxiliary qubits and transform the measurement to an integer value $y$.
7. Compute the angle using: $\lambda = \frac{y}{2^m}$

For executing this workflow the **run** method of the class should be executed. The corresponding measurements are stored as a pandas DataFrame into the class attribute **result** which has the following columns:

* **States**: Possible quantum states of the auxiliary qubits
* **Int_lsb**:  conversion from the quantum state to an integer following **lsb** (bit farthest to the right will be least significant)
* **Probability**: Computed frequency of the quantum state.
* **Int**: conversion from the quantum state to an integer (the bit farthest to the right will be most significant)
* **lambda**: is the estimated obtained phase.

Depending on the values of the input variables *shots* and *complete* the results showed in the **result** DataFrame can contain all the possible quantum states or only the measured states.

In [ ]:
qft_pe.run()

In [ ]:
c = qft_pe.circuit
%qatdisplay c --svg

In [ ]:
qft_pe.result

In the case of the used *Qiskit* example:

$$i\frac{\pi}{2} = 2 \pi i \lambda$$

So the desired solution will be: $\lambda = \frac{1}{4}$

In the before case we have fixed the *complete* variable to **True**, so all the possible states are returned. In this case the $\lambda$ value will be the one with the higher probability: $\lambda = 0.25$


In [ ]:
qft_pe.result.sort_values('Probability', ascending=False).iloc[0]['lambda']

If *shots* is different of zero and  *complete* is set to **False** only the measured states will be returned!!

In [ ]:
auxiliar_qbits_number = 2
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : auxiliar_qbits_number,
    'complete': False,
    'shots':100
}
qft_pe = CQPE(**qft_pe_dict)
qft_pe.run()
qft_pe.result

Addtitionally the attribute **quantum_times** measures the time for the quantum part of the algorithm (or simulated)

In [ ]:
qft_pe.quantum_times

## Another example

We are going to reproduce now the Qiskit example under the section *IPE example with a 2-qubit gate* (in https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb). 

In this case, they use 2 qubits and want to estimate the phase for a unitary operator $\mathcal{cT}$ operator. This operator adds a $\frac{\pi}{4}$ phase to state $|11\rangle$ and leaves unchanged other states. 

In this case:

$$\mathcal{cT} |11\rangle = e^{i\frac{\pi}{4}} |11\rangle$$

So:

$$i\frac{\pi}{4} = 2 \pi i \lambda$$

So $\lambda = \frac{1}{8}$

In [ ]:
#create initial state
n_qbits = 2
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state --svg

In [ ]:
#Create cT operator
unitary_operator = qlm.QRoutine()
uq_qbits = unitary_operator.new_wires(n_qbits)
unitary_operator.apply(qlm.PH(np.pi/4.0).ctrl(), 0, 1)

In [ ]:
%qatdisplay unitary_operator --svg

In [ ]:
#now PE with QFT!!
n_cbits = 3
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : n_cbits,  
    'shots': 100,
    'complete': False
}
qft_pe = CQPE(**qft_pe_dict)

In [ ]:
qft_pe.run()

In [ ]:
qft_pe.result.sort_values('Probability', ascending = False)

The result for $\varphi=0.125$ that is the value obtained in the Qiskit example.

## 3. Application to Amplitude Estimation

The problem of *Amplitude Estimation* is the following: given an oracle operator $\mathcal{A}$ with the following behaviour:

$$|\Psi\rangle= \mathcal{A}|0\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $\sqrt{a}$. One naive solution will be measuring $N$ times and getting the probabilities of obtaining $|\Psi_0\rangle$. In this case the error in the estimation of $\sqrt{a}$ will scale with: $\epsilon_{a} \sim \frac{1}{\sqrt{N}}$. Usually, it is useful to express this error in function of the number of oracle calls: $N_{\mathcal{A}}$. In this naive case, it can be demonstrated that:

$$\epsilon_{a} \sim \frac{1}{\sqrt{N_{\mathcal{A}}}}$$

Classical *Phase Estimation* algorithm with *QFT* can be used for obtaining a quadratic speed up in *Amplitude Estimation* problems. 

To apply this algorithm to *Amplitude Estimation* problems following steps should be done:

1. Doing the following substitution:

$$\sqrt{a} = \sin\left(\theta\right)$$

2. The action of the oracle operator will be now:

$$|\Psi\rangle= \mathcal{A}|0\rangle  = \sqrt{a} |\Psi_0\rangle  + \sqrt{1-a}|\Psi_1\rangle = \sin\left(\theta\right) |\Psi_0\rangle + \cos\left(\theta\right) |\Psi_1\rangle \tag{1}$$

3. Creation of the Grover-like operator of the oracle operator $\mathcal{A}$:

$$\mathbf{G}(\mathcal{A}) = \mathcal{A} \left(\hat{I} - 2|0\rangle\langle 0|\right) \mathcal{A}^{\dagger}\left(\hat{I} - 2|\Psi_0\rangle\langle \Psi_0|\right)$$

Now the classical *Phase Estimation* algorithm can be used straightforward by doing the following substitutions:

$$
    \begin{array}{l}
    & |\Psi\rangle \longrightarrow \mathcal{A}|0\rangle \\
    & \mathcal{Q} \longrightarrow \mathbf{G}(\mathcal{A})
    \end{array}
$$

It can be demonstrated that the $\mathbf{G}(\mathcal{A})$ operator can be expressed as a rotation in the plane of an angle $\theta$, this is:

$$
\mathbf{G}(\mathcal{A}) = 
\begin{pmatrix}
\cos 2\theta & -\sin 2\theta\\
\sin 2\theta & \cos 2\theta \\
\end{pmatrix}
$$

So the autovalues of the $\mathbf{G}(\mathcal{A})$ will be $e^{\pm i2\theta}$

So using our phase estimation convention:

$$i2\theta = 2 \pi i \lambda$$

We arrive to: $\theta = \pi \lambda$.

It can be demostrated that using the classical *Phase Estimation* algorithm the error of the estimation in $\theta$ (and in $\sqrt{a}$) will scale as:

$$\epsilon_{\theta} \sim \frac{1}{N_{\mathcal{A}}}$$

So a quadratic speed up is obtained with respect to a naive solution!!

### 3.1 Using *CQPE* class for *Amplitude Estimation*

In this section, we explain how to use the **CQPE** class for solving *Amplitude Estimation* problems. 

First, we will define the following amplitude estimation problem:

$$|\Psi\rangle = \mathcal{A}|0\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right] \tag{2}$$

So comparing (2) with (1):

$$\sqrt{a}|\Psi_0\rangle = \sin(\theta)|\Psi_0\rangle = \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle$$

and 

$$\sqrt{1-a}|\Psi_1\rangle = \cos(\theta)|\Psi_1\rangle = \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

Following cells creates the initial state $|\Psi\rangle = \mathcal{A}|0\rangle$

In [ ]:
from QQuantLib.DL.data_loading import load_probability
from QQuantLib.AA.amplitude_amplification import grover

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

Now we need to compute the Grover-like operator from the oracle operator $\mathcal{A}$. The functions from *AA* module will be used (see notebook **02_Amplitude_Amplification_Operators.ipynb** for more information)

In [ ]:
target = [0, 0, 1]
index = range(oracle.arity)
grover_gate = grover(oracle, target, index)
#Comment before line and uncomment following for For multiplexor implementation of multi-controlled Z gate
#grover_gate = grover(oracle, target, index, mcz_qlm=False)
%qatdisplay grover_gate --depth 2 --svg

Now that we have translated our amplitude amplification problem to an phase estimation problem we proceed to use our class normally. We provide the *oracle* as the **initial_state** and the correspondent Grover-like operator as the **unitary_operator**. Additionally auxiliar_qbits_number for estimating the phase should be provided.

In [ ]:
n_cbits = 8
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': oracle,
    'unitary_operator': grover_gate,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : n_cbits,  
    'shots': 1000,
    'complete':True #For getting all the posible 2^n_cbits states
}
qft_pe = CQPE(**qft_pe_dict)
qft_pe.run()

In [ ]:
#Using the circuit attribute for draw the complete routine
c = qft_pe.circuit
%qatdisplay c --svg

In [ ]:
#We can plot the obtained the frequency vs the theta
qft_pe.result.sort_values('lambda', inplace=True)
plt.plot(qft_pe.result['lambda'], qft_pe.result['Probability'], 'o-')
plt.xlabel(r'$\lambda$')
plt.ylabel('Probability')

Now we can compute $\theta = \pi \lambda$.

In [ ]:
qft_pe.result['theta'] = np.pi * qft_pe.result['lambda']

In [ ]:
#We can plot the obtained the frequency vs the theta in the [0, pì/2] range
qft_pe.result.sort_values('theta', inplace=True)
plt.plot(qft_pe.result['theta'], qft_pe.result['Probability'], 'o-')
plt.xlabel(r'$\theta$')
plt.ylabel('Probability')

In this case, the 2 obtained maximums reflect the 2 *eigenvalues* of $\mathbf{G}(\mathcal{A})$: $\pm\theta$. In general it is useful to restrict $\theta$ to the interval $(0, \frac{\pi}{2}$). This can be done in the following cell:

In [ ]:
qft_pe.result['theta_90'] = qft_pe.result['theta']
qft_pe.result['theta_90'].where(
    qft_pe.result["theta_90"] < 0.5 * np.pi,
    np.pi - qft_pe.result["theta_90"],
    inplace=True,
)

In [ ]:
#We can plot the obtained the frequency vs the theta in the [0, pì/2] range
qft_pe.result.sort_values('theta_90', inplace=True)
plt.plot(qft_pe.result['theta_90'], qft_pe.result['Probability'], 'o-')
plt.xlabel(r'$\theta$')
plt.ylabel('Probability')
plt.title(r'$\theta \in [0, \frac{\pi}{2}]$')

Finally, we need to compute the desired value of $a$. Following the equations presented in the notebook this can be done by $a =\sin(\theta)^2$.

**In our implementation of the QPE the convention is changed and the following relationship MUST BE used**:

$$a = \cos(\theta)^2$$

In [ ]:
qft_pe.result['a']=np.cos(qft_pe.result['theta_90'])**2

In [ ]:
#We can plot the obtained a vs the theta in the [0, pì/2] range
plt.plot(qft_pe.result['a'], qft_pe.result['Probability'], 'o-')
plt.xlabel(r'$a$')
plt.ylabel('Probability')
plt.title(r'$a \in [0, 1]$')

### What *a* should be returned?

From the point of view of the **QPE** algorithm for **AE** one possible solution would be to return the most frequently $a$ measured. But from an actual computer perspective this question is tricky because the computer used could not be perfect (actual quantum computers are far away from being good) and the **QPE** algorithm can not be executed properly and the results can be different from the expected result of the bare algorithm. 

We are going to use the mean of the probability distribution obtained from the measures. This is:

$$\tilde{a} = \int a * p(a) * da = \sum_{i} a[i] * P(a[i])$$

where $p(a)$ will be the probability of obtaining $a$ (experimentally this will be the number of times the $a$ value was obtained divided by the number of shots executed).

In [ ]:
qft_pe.result.sort_values('Probability', inplace=True, ascending=False)
mean_a = qft_pe.result['Probability'] @ qft_pe.result['a']

In [ ]:
print("Most frequent value of a: {}".format(qft_pe.result['a'].iloc[0]))
print("Mean value of a: {}".format(mean_a))
print("Classical result: ",probability[1])
print('Test OK. Most Frequent Value: ', probability[1]- qft_pe.result['a'].iloc[0] < 0.005)
print('Test OK. Mean Value: ', probability[1] - qft_pe.result['Probability'] @ qft_pe.result['a'] < 0.005)

### Asociated error of *a*

Additionally, we can compute an error associated with the estimator of $\tilde{a}$, $\epsilon_a$. For the **QPE** algorithm in an ideal quantum computer the error will depend on the number of auxiliary qubits, $m$:

$$\epsilon_a^{QPE} \lt \frac{2\pi}{2 ^ m}$$

But in an actual quantum computer, the errors will be greater. For this case, the most direct way for computing it will be to use the square root of the variance of the probability density measured (this will be the experimental error):

$$\epsilon_a^{P(a)} = \sqrt{\int p(a) (a-\tilde{a}) ^2 * da} = \sqrt{\sum_{i} (a[i] -\tilde{a})^2 * P(a[i])}$$

In order to deal with all the situations, a perfect quantum computer (i.e. noiseless simulation) or an actual quantum computer the best approach is taking the maximum of these two errors:

$$\epsilon_a = \max (\epsilon_a^{QPE}, \epsilon_a^{P(a)} )$$

In [ ]:
eps_a_pa = (qft_pe.result['Probability'] @ ((qft_pe.result['a'] - mean_a) ** 2))**0.5
eps_qpe = 2 * np.pi / 2 ** qft_pe.auxiliar_qbits_number
epsilon = max([eps_a_pa, eps_qpe])
print('QPE error: {} . Probability Density Error: {}'.format(eps_qpe, eps_a_pa))
print('Total Error: {}'.format(epsilon))

## 4. Amplitude Estimation with classical Quantum Phase Estimation

In order to simplify **Amplitude Estimation** calculations done with the **CQPE** class (see Section 3) the **CQPEAE** (stands for **C**lassical **Q**uantum **P**hase **E**stimation **A**mplitude **Estimation**) class was created. The idea behind this class is dealing with the initial overheating shown in Section 3 for using **CQPE** in a straightforward way for *Amplitude Estimation* calculations. The class is under the module **ae_classical_qpe.py** of the *Amplitude Estimation* package of the *QQuantLib* library  (**QQuantLib/AE/ae_classical_qpe**)

**Note**

The idea is that the **CQPEAE** have a similar scheme like the **MLAE class** from **QQuantLib/AE/maximum_likelihood_ae** module

To create **CQPEAE** associated object following inputs are mandatory:

1. Oracle: QLM AbstractGate or QRoutine with the implementation of the Oracle for creating the Grover operator.
2. target: this is the marked state in binary representation as a Python list
3. index: list of the qubits affected by the Grover operator.

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
#Here we created the mandatory oracle
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

#This will be the target state for grover and the list of qubits affected by Grover operator
target = [0, 0, 1]
index = range(oracle.arity)

Now we have all mandatory inputs so we created the *CQPEAE* object. Following **MLAE class** convention other parameters can be provided with a Python dictionary. In this case, the following keys can be provided:

* *auxiliar_qbits_number*: number of qubits for doing phase estimation (default: 8)
* *shots*: number of shots (default: 100)
* *qpu*: qpu solver
* mcz_qlm: for using QLM multi-controlled Z gate (True, default) or using multiplexor implementation (False)

In [ ]:
from QQuantLib.AE.ae_classical_qpe import CQPEAE

In [ ]:
ae_pe_qft_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 4,
    'shots': 1000,
    'mcz_qlm': False    
}

ae_pe_qft = CQPEAE(
    oracle=oracle,
    target=target,
    index=index, 
    **ae_pe_qft_dict
)

When instantiated the *CQPEAE* class the *Grover* operator associated to the oracle operator is created. It can be access using attribute *_grover_oracle*.

In [ ]:
grover_circ = ae_pe_qft._grover_oracle
%qatdisplay grover_circ --depth 2 --svg

Now the *ae_pe_qft* object has all mandatory inputs for calling the *CQPE* class and executing the algorithm. The **run** method from the **CQPEAE** class does this step. Additionally when the *run* method is executed following properties from the **CQPEAE** class are populated:

* *cqpe*: this property is an object from the **CQPE** class that was initialized with corresponding arguments created by the **CQPEAE** class.
* *final_results*: this is the final_results obtained from the *pe_qft* method of the property object *cqpe* (see section 2.3).
* *theta*: estimated $\theta$ obtained from **CQPE** algorithm (used the meanb of the probability distribution)
* *ae* amplitude estimated solution based on  $a=cos^2 \theta$
* *run_time*: elapsed time of a complete execution of the **run** method

Additionally, the different computed errors for $a$ can be accessed with the following attributes:

* *epsilon_qpe* : error due  to the **QPE** algorithm
* *epsilon_prob*: statistical error of the measurements performed
* *epsilon*: final error resulting in the maximum of the two aforementioned errors.


In [ ]:
a_estimated = ae_pe_qft.run()

In [ ]:
print('a_estimated = ',a_estimated) 
print('ae_pe_withQFT.theta= ', ae_pe_qft.final_results)
print('ae_pe_withQFT.theta= ', ae_pe_qft.theta)
print('ae_pe_withQFT.a= ', ae_pe_qft.ae)
print("Classical result: ",probability[1])

print('QPE error: ', ae_pe_qft.epsilon_qpe)
print('Experimental error: ', ae_pe_qft.epsilon_prob)
print('Final error: ', ae_pe_qft.epsilon)

In [ ]:
print('Test OK: ',np.abs(ae_pe_qft.ae-probability[1])< 0.005)

In [ ]:
print('Elapsed time for run method: ',ae_pe_qft.run_time)

Of course the configuration of the algorithm can be changed by setting the attributes of the class an calling the *run* method again!!

In [ ]:
print(ae_pe_qft.auxiliar_qbits_number)

In [ ]:
ae_pe_qft.auxiliar_qbits_number = 10
print(ae_pe_qft.auxiliar_qbits_number)

In [ ]:
#run method again
a_estimated = ae_pe_qft.run()

In [ ]:
print('a_estimated = ',a_estimated) 
print('ae_pe_withQFT.theta= ', ae_pe_qft.theta)
print('ae_pe_withQFT.a= ', ae_pe_qft.ae)
print("Classical result: ",probability[1])

print('QPE error: ', ae_pe_qft.epsilon_qpe)
print('Experimental error: ', ae_pe_qft.epsilon_prob)
print('Final error: ', ae_pe_qft.epsilon)

In [ ]:
print('Elapsed time for run method: ',ae_pe_qft.run_time)

As explained before, the **cqpe** attribute from *CQPEAE* class is an object created from class **CQPE** so all the methods and attributes of this class can be accessed using the **cqpe** attribute (**NOT RECOMMENDED**).

We can access to the quantum circuit used by the **CQPE**

In [ ]:
#First we access to the attribute they stores the CQPE object
attribute_pewqtf = ae_pe_qft.cqpe
#Then we take the circuit attribute from the PhaseEstimationwQFT object (see above in the notebook)
c=attribute_pewqtf.circuit
%qatdisplay c --svg --depth 0

When the *run* method is executed following class attributes are populated:

* *circuit_statistics*: python dictionary with the complete statistical information of the circuit.
* *oracle_calls*: number of total oracle calls for a complete execution of the algorithm
* *max_oracle_depth*: maximum number of applications of the oracle for the complete execution of the algorithm.

In [ ]:
ae_pe_qft.circuit_statistics

In [ ]:
#Total number of oracle calls
print("The total number of the oracle calls for the CQPEAE was: {}".format(ae_pe_qft.oracle_calls))

In [ ]:
#Number of maximum oracle applications
ae_pe_qft.max_oracle_depth

In [ ]:
ae_pe_qft.quantum_times

In [ ]:
ae_pe_qft.quantum_time